# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В качестве исходных данных предоставлена информация о клиенте. **Информация о данных в таблице**:
<ul>
    <li>children — количество детей в семье</li>
    <li>days_employed — трудовой стаж в днях</li>
    <li>dob_days — возраст клиента в годах</li>
    <li>education — образование клиента</li>
    <li>education_id — идентификатор образования</li>
    <li>family_status — семейное положение</li>
    <li>family_status_id — идентификатор семейного положения</li>
    <li>gender — пол клиента</li>
    <li>income_type — тип занятости</li>
    <li>debt — имел ли задолженность по возврату кредитов</li>
    <li>total_income — доход в месяц</li>
    <li>purpose — цель получения кредита</li>
</ul>

## Шаг 1. Откройте файл с данными и изучите общую информацию

Считаем файл с дынными и выведем первые 25 строк.

In [2]:
import pandas as pd

clients_data = pd.read_csv('/datasets/data.csv')
clients_data.head(25)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


После первичного осмотра данных сразу видны следующие проблемы:
<ul>
    <li>В столбце с трудовым стажем 'days_employed' есть как отрицательные, так и положительные значения, необходимо понять почему они такие</li>
    <li>В столбце с трудовым стажем 'days_employed' есть значения NaN</li>
    <li>В столбце с уровнем образования 'education' часть данных написана с верхнем регистре, например 'ВЫСШЕЕ'</li>
    <li>В столбце с уровнем доходов 'total_income' также есть значения NaN. Возможно, есть зависимость со значением NaN в столбце с трудовым стажем 'days_employed', так как в выведенных строках есть строка с NaN значениями в обоих столбцах</li>
    <li>В столбце с целью получения кредита видны значения, схожие по смыслу, но по разному написанные, например "покупка жилья", "операции с жильём" и "покупка жилья для семьи"</li>
</ul>    

Переименуем некоторые сдолбцы, для лучшего обозначения данных, хранящихся в этом слобце:
<ul>
    <li>'income_type' -> 'work_type' - столбец хранит тип занятости человек</li>
    <li>'total_income' -> 'monthly_income' - столбец хранит ежемесячный доход</li>
    <li>'debt' -> 'credit_debt' - столбец хранит информацию имел ли клиент задолженность по кредиту</li>
</ul>
Выведем названия стобцов, что бы убедится в смене их названия

In [3]:
clients_data = clients_data.rename({'income_type': 'work_type', 'total_income': 'monthly_income', 'debt': 'credit_debt'}, axis='columns')
clients_data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'work_type',
       'credit_debt', 'monthly_income', 'purpose'],
      dtype='object')

Выведем информацию о таблице.

In [4]:
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
work_type           21525 non-null object
credit_debt         21525 non-null int64
monthly_income      19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Как было предположенно**, количество строк со значением NaN в слолбце с **общим рабочим стажем** равно количеству строк со значением NaN с **доходом в месяц**.<br>
Возможно, это связано с тем, что клиент не успел подать данные как о своём рабочем стаже, так и о доходе. Возможно, эти люди только устроились на работу. 
Посчитаем количество таких строк.

In [5]:
null_count = clients_data[(clients_data['days_employed'].isnull()) & (clients_data['monthly_income'].isnull())]['children'].count()
total_count = clients_data['children'].count()
null_percent = null_count / total_count
print('Количество строк с NaN значениями:', null_count)
print('Процент строк с NaN значениями: {:.1%}'.format(null_percent))

Количество строк с NaN значениями: 2174
Процент строк с NaN значениями: 10.1%


Null данных достаточно много, 2174 строк. Это 10.1% от общего колоичества данных.
Избаляться от такого количества данных нельзя, так как мы потеряем слишком много информации.

В качестве решения данной проблемы рассмотрим два метода:
<ul>
    <li>Заменим все значения на 0. Такой подход менее информативен, так как мы лишимся 10% данных и такой подход почти идентичен удалению строк с данными.</li>
    <li>Заменим все значения на средние значения, учитывая: возраст клиента и тип занятости клиента. Этот подход лучше, так как мы не лишаем себя данных.</li>
</ul>
Учитывая описанное, остановимся на варианте с заменой NaN значений на средние, с учётом возраста клиента и типа занятости.

Исследуем значения различных столбцов данных:
<ul>
    <li>Количество детей</li>
    <li>Уровень образования</li>
    <li>Пол клиента</li>
    <li>Тип занятости</li>
    <li>Цель получения кредита</li>
</ul>

In [6]:
print(clients_data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В данных есть ошибочные данные: -1 ребёнок и 20 детей. Скорее всего, при заполнении данных была допущена ошибка. Заменим в будущем эти значения на 1 и 2.<br>
С другой стороны, значение в 20 детей может быть ошибкой работы формы для заполнения данных или же его особенностью.<br>
В дальнейшем, планируется разбить количество детей на диапазон, 0 - бездетные, 1-2 - семья с детьми, 3 и больше - многодетные семьи. В таком случае, если строкам с 20 детьми присвоить значение 2, то мы потеряем большое количество данных для "многодетных семей", тогда нельзя их списывать со счетов и возможно стоит просто приравнять их к значению "многодетные семьи"

In [7]:
print(clients_data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


**Изначальное предположение подтвердилось**. Видно много ошибок. Будет необходимо привести столбец с данными об образовании к нижнему регистру.

In [8]:
print(clients_data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Одна из строк данных имеет значение **XNA** отличное от остальных. Не будем ничего делать с этим, так как это всего 1 строка из всех и на финальный результат она не повлияет.

In [9]:
print(clients_data['work_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


С этим столбцом всё в порядке. Нет ни дубликатов, ни ошибочных данных

In [10]:
print(clients_data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**И это предположение также оказалось верным**. В этом столбце есть значения, схожие по смыслу, однако по разному написанные, например: "свадьба", "на проведение свадьбы", "сыграть свадьбу" и т.д. Необходимо объединить такие данные. 

Также проверим, есть ли данные о возврасте, равном нулю.

In [11]:
print(clients_data[clients_data['dob_years'] == 0]['dob_years'].value_counts())

0    101
Name: dob_years, dtype: int64


В таблице есть строки, в которых возраст указан равным нулю, чего быть не должно. С одной стороны - количество таких данных очень маленькое и не повлияет на финальный результат, если их удалить. С другой стороны, можно заменить этот возраст на среднее значение по стажу.

**Вывод**<br>
Проведя первичный анализ данных, были обнаружены следующие ошибки:
<ul>
    <li>Необходимо исправить значения столбца о рабочем стаже</li>
    <li>Избавиться от NaN значений в столбцах о рабочем стаже и ежемесячном доходе</li>
    <li>В данных о детях есть значения -1 и 20. Их необходимо исправить</li>
    <li>В столбце о возрасте есть нулевые значения</li>
    <li>В столбце с образованием много ошибок в формате написания, необходимо избавиться от них</li>
    <li>Необходимо свести категории целей кредита к единому списку, так как на данном этапе много схожих по смыслу значений, но написанных разными словами</li>
</ul>

## Шаг 2. Предобработка данных

### Обработка пропусков

Во время анализа данных были обнаружены NaN значения в столбцах с трудовым стажем и доходом.<br>
Для заполнения этих пропусков было решено заменить эти значения на средние значения с учётом типа занятости и возраста.

#### Отрицательные и очень большие значения стажа

В данных присутствуют отрицательные значения и слишком большие положительные значения.<br>
Отрицательные значения скорее всего связаны с тем, что так как человек на данный момент работает, то отсутствует дата увольнения с последнего места работы, что в свою очередь приводит к отрицательному результату.<br>
Для начала посмотрим средние значения по стажу по разным типам занятости, а так же количество положительных значений.<br>
Напишем функцию, которая принимает на вход DataFrame, и выводит инфорамцию о количестве записей, среднем стаже и количестве записей с положительным стажем.<br>
Вызовем функцию для данных о клиентах и посмотрим полученную инфорамцию

In [12]:
def group_days_employed(data):
    grouped_data = clients_data.groupby('work_type').agg({'days_employed': ['count', 'mean']})
    grouped_data = grouped_data.rename({'count': 'Количество строк', 'mean': 'Среднее'}, axis='columns')
    positive_data = clients_data[clients_data['days_employed'] > 0].groupby('work_type')
    positive_data = positive_data['work_type'].count()
    grouped_data['Положительных значений'] = positive_data
    return grouped_data

In [13]:
grouped_data = group_days_employed(clients_data)
print(grouped_data)

                   days_employed                Положительных значений
                Количество строк        Среднее                       
work_type                                                             
безработный                    2  366413.652744                    2.0
в декрете                      1   -3296.759962                    NaN
госслужащий                 1312   -3399.896902                    NaN
компаньон                   4577   -2111.524398                    NaN
пенсионер                   3443  365003.491245                 3443.0
предприниматель                1    -520.848083                    NaN
сотрудник                  10014   -2326.499216                    NaN
студент                        1    -578.751554                    NaN


Положительные данные находятся только в данных о пенсионерах и безработных. Все остальные значения заменим на положительтные, взяв исходное значение по модулю.

In [14]:
clients_data['days_employed'] = clients_data['days_employed'].abs()

Проверим результат замены и убедимся в отсутствии отрицательных значений.

In [15]:
grouped_data = group_days_employed(clients_data)
print(grouped_data)

                   days_employed                Положительных значений
                Количество строк        Среднее                       
work_type                                                             
безработный                    2  366413.652744                      2
в декрете                      1    3296.759962                      1
госслужащий                 1312    3399.896902                   1312
компаньон                   4577    2111.524398                   4577
пенсионер                   3443  365003.491245                   3443
предприниматель                1     520.848083                      1
сотрудник                  10014    2326.499216                  10014
студент                        1     578.751554                      1


Все значения положительные.<br>
Природу появления слишком больших значений понять не удалось, однако, все эти значения принадлежат пенсионерам и безработным.<br>
Для решения этой особенности данных, будем приводить все значения трудового стажа к диапазонам и всем пенсионерам присвоим с таким большим стажем категорию '>50 лет'.<br>
С безработными всего 2 значения в таблице, по этому от этих данных мы можем избавиться, не жертвуя при этом большим количеством данных.<br>
Кроме того, безработным вряд ли будут выдавать кредит.

In [16]:
clients_data = clients_data[clients_data['work_type'] != 'безработный'].reset_index(drop=True)
grouped_data = group_days_employed(clients_data)
print(grouped_data)

                   days_employed                Положительных значений
                Количество строк        Среднее                       
work_type                                                             
в декрете                      1    3296.759962                      1
госслужащий                 1312    3399.896902                   1312
компаньон                   4577    2111.524398                   4577
пенсионер                   3443  365003.491245                   3443
предприниматель                1     520.848083                      1
сотрудник                  10014    2326.499216                  10014
студент                        1     578.751554                      1


#### Обработка NaN значений в стаже и заработке

Теперь необходимо заполнить строки, в которых есть NaN значения.<br>
Такие значения нашлись в двух столбцах: общий стаж работы и месячный заработок. Заменим на среднее значение и медиану при той же должности и образовании. Так как в столбце с образованием есть ошибки, связанные с названиями, на данном этапе возьмём значение из столбца education_id.<br>
Для этого сделаем следующие:
<ul>
    <li>Применим к столбцу со стажем применим группировку 'groupby' по столбцам с типом занятости и id образования, затем заполнение NaN значений 'fillna' средним значением 'mean' по столбцу со стажем работы</li>
    <li>Отсортируем значения ежемесячного дохода для подсчёта медианы</li>
    <li>Применим к столбцу с ежемесячным доходом применим группировку 'groupby' по столбцам с типом занятости и id образования, затем заполнение NaN значений 'fillna' медианой 'median' по столбцу с ежемесячным заработком</li>
</ul>

In [17]:
clients_data['days_employed'] = clients_data.groupby(['work_type', 'education_id'])['days_employed'].fillna(clients_data.groupby(['work_type', 'education_id'])['days_employed'].transform('mean'))
clients_data = clients_data.sort_values(by='monthly_income')
clients_data['monthly_income'] = clients_data.groupby(['work_type', 'education_id'])['monthly_income'].fillna(clients_data.groupby(['work_type', 'education_id'])['monthly_income'].transform('median'))
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 14584 to 21508
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       21523 non-null float64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
work_type           21523 non-null object
credit_debt         21523 non-null int64
monthly_income      21523 non-null float64
purpose             21523 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.8+ MB


Теперь NaN данных в таблице нет.

### Обработка ошибочных значений

Обработаем непонятные значения

#### Отрицательное количество детей
Как было описано в предыдущем этапе, были обнаружены значения в количестве детей, равные -1 и 20. С это ошибкой было принято следующее:
<ul>
    <li>Значения -1 заменим на 1, так как мы предполагаем, что была допущена ошибка при вводе</li>
    <li>Значение 20 мы не трогаем, а при дальнейшем анализе присваиваем таким данным категорию "многодетная семья"</li>
</ul>

In [18]:
clients_data['children'] = clients_data['children'].replace(-1, 1)
print(clients_data['children'].value_counts())

0     14148
1      4864
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


В таблице больше нет строк с отрицательным числом детей.

#### Нулевой возраст клиентов

Предположим, что клиенты не указали свой возраст. Избавимся от таких данных, как мы избавлялись до этого от данных о безработных клиентах. Так как этих данных всего 101 запись, мы не потеряем большого количества информации.<br>
Проверим после этого не осталось ли данных с нулевым возрастом клиента.

In [19]:
clients_data = clients_data[clients_data['dob_years'] != 0].reset_index(drop=True)
print(clients_data[clients_data['dob_years'] == 0]['dob_years'].value_counts())
clients_data.info()

Series([], Name: dob_years, dtype: int64)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21422 entries, 0 to 21421
Data columns (total 12 columns):
children            21422 non-null int64
days_employed       21422 non-null float64
dob_years           21422 non-null int64
education           21422 non-null object
education_id        21422 non-null int64
family_status       21422 non-null object
family_status_id    21422 non-null int64
gender              21422 non-null object
work_type           21422 non-null object
credit_debt         21422 non-null int64
monthly_income      21422 non-null float64
purpose             21422 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Нетипичное написание уровня образования клиента

Как мы выяснили на этапе первичного анализа данных, в столбце с уровнем образования клиентов есть множество данных, написанных отлично друго от друга, например "ВЫСШЕЕ" и "высшее".<br>
Избавимся от этой проблемы: переведём всё в нижний регистр. После этого посмотрим ещё раз на уникальные значения этого столбца, что бы убедиться в том, что ошибка исправлена.

In [20]:
clients_data['education'] = clients_data['education'].str.lower()
print(clients_data['education'].unique())

['среднее' 'начальное' 'высшее' 'неоконченное высшее' 'ученая степень']


Теперь все значения столбца с образованием написаны в нижнем регистре

#### Семейное положение
Также было замечено, что в столбце о семейном положении есть столбцы "Не женат...", заменим также на нижний регистр для однотипности записей

In [21]:
clients_data['family_status'] = clients_data['family_status'].str.lower()
print(clients_data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'не женат / не замужем' 'в разводе'
 'вдовец / вдова']


**Вывод**<br>
Итак, мы избавились от ошибочных значений: разобрались с непонятными значениями в количестве детей, указанным нулевым возрастом и с написанием уровня образования.

### Замена типа данных

Посмотрим ещё раз информацию о столбцах таблицы

In [22]:
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21422 entries, 0 to 21421
Data columns (total 12 columns):
children            21422 non-null int64
days_employed       21422 non-null float64
dob_years           21422 non-null int64
education           21422 non-null object
education_id        21422 non-null int64
family_status       21422 non-null object
family_status_id    21422 non-null int64
gender              21422 non-null object
work_type           21422 non-null object
credit_debt         21422 non-null int64
monthly_income      21422 non-null float64
purpose             21422 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Столбцы, хранящие в себе общий трудовой стаж и месячный доход по какой то причине содержат вещественные данные. В данной ситуации, точность в значениях не нужна. Приведём эти данные к целочисленному типу.<br>
Так как данные представлены в вещественном виде, для перевода в целочисленный будем использовать функцию **astype()**, указав в качестве параметра строку **int**.<br>
После смены типов данных вызовем ещё раз информацию о таблице, чтобы убедится в смене типов.

In [23]:
clients_data['days_employed'] = clients_data['days_employed'].astype('int')
clients_data['monthly_income'] = clients_data['monthly_income'].astype('int')
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21422 entries, 0 to 21421
Data columns (total 12 columns):
children            21422 non-null int64
days_employed       21422 non-null int64
dob_years           21422 non-null int64
education           21422 non-null object
education_id        21422 non-null int64
family_status       21422 non-null object
family_status_id    21422 non-null int64
gender              21422 non-null object
work_type           21422 non-null object
credit_debt         21422 non-null int64
monthly_income      21422 non-null int64
purpose             21422 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Выведем часть таблицы и посмотрим на изменённые данные.

In [24]:
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose
0,0,359219,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость
1,0,369708,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием
2,1,3642,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля
3,0,359726,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы
4,0,346602,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость
5,0,347356,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью
6,0,370478,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844,покупка жилья для семьи
7,0,340952,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью
8,0,4959,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование
9,1,354563,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование


**Вывод**

Теперь все числовые данные в таблице хранятся в целочисленном формате. В случае с трудовым стажем и месячным доходом такой формат данных подходит лучше, так как не требует точности.

### Обработка дубликатов

Посмотрим, есть ли в таблице дубликаты функцией **duplicated()** и посчитаем их количество, добавив функцию **sum()**.

In [25]:
print('Количество строк - дубликатов в таблице:', clients_data.duplicated().sum())

Количество строк - дубликатов в таблице: 71


Да, в таблице найден 71 дубликат:

In [26]:
clients_data[clients_data.duplicated(keep=False)].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose
20985,0,2018,54,высшее,0,женат / замужем,0,M,компаньон,0,201785,операции с коммерческой недвижимостью
19475,0,2018,54,высшее,0,женат / замужем,0,M,компаньон,0,201785,операции с коммерческой недвижимостью
20147,0,2018,38,высшее,0,гражданский брак,1,F,компаньон,0,201785,на проведение свадьбы
21188,0,2018,38,высшее,0,гражданский брак,1,F,компаньон,0,201785,на проведение свадьбы
21023,1,2235,40,среднее,1,гражданский брак,1,F,компаньон,0,159070,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21412,0,365007,54,среднее,1,женат / замужем,0,F,пенсионер,0,114842,операции с жильем
19404,0,365176,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,144240,дополнительное образование
20075,0,365176,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,144240,дополнительное образование
20005,0,365176,64,высшее,0,гражданский брак,1,F,пенсионер,0,144240,на проведение свадьбы


Возможно, дубликаты появились после сведения уровня образования к нижнему регистру и в каких то строках. Возможно, дубликаты появились в результате ошибок системы.<br>
Избавимся от дубликатов функцией **drop_duplicated()**, вызвав за ней функцию **reset_index** с параметром **drop=True** чтобы пересчитать индексы строк. Выведем количество дубликатов в обновлённой таблицы.

In [27]:
clients_data = clients_data.drop_duplicates().reset_index(drop=True)
print('Количество строк - дубликатов в таблице:', clients_data.duplicated().sum())

Количество строк - дубликатов в таблице: 0


**Вывод**

В таблице были найдены дубликаты, которые возможно появились в результат в результате обработки написания уровня образования клиента, а возможно в результате некорректной работы системы заполнения данных.<br>
Дубликаты были удалены из таблицы.

### Лемматизация

В таблице с данными, в столбце с целью кполучения кредита было обнаружено много значений, схожих по смыслу, но по разном уописанных. Посмотрим на них ещё раз:

In [28]:
print(clients_data['purpose'].value_counts())

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   648
операции с коммерческой недвижимостью     648
операции с жильем                         646
покупка жилья                             640
жилье                                     640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              604
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

#### Выделение лемм
Для решения данной проблемы выделим из всего этого списка леммы и в будущем составим список, по которому будем классифицировать цель получения кредита однозначно. Подключим библиотеку pymystem3 и Counter из библиотеки collections.

In [29]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 

Посмотрим список лемм стобца с целью получения кредита.

In [30]:
all_lemmas = []

for row_purpose in clients_data['purpose']:
    lemmas = m.lemmatize(row_purpose)
    all_lemmas.extend(lemmas)
    
lemmas_counts = Counter(all_lemmas)
print(lemmas_counts)

Counter({' ': 33431, '\n': 21351, 'недвижимость': 6328, 'покупка': 5869, 'жилье': 4434, 'автомобиль': 4284, 'образование': 3995, 'с': 2904, 'операция': 2593, 'свадьба': 2310, 'свой': 2223, 'на': 2210, 'строительство': 1873, 'высокий': 1366, 'получение': 1309, 'коммерческий': 1306, 'для': 1285, 'жилой': 1224, 'сделка': 938, 'дополнительный': 902, 'заниматься': 900, 'проведение': 764, 'сыграть': 760, 'сдача': 648, 'семья': 637, 'собственный': 633, 'со': 627, 'ремонт': 604, 'подержанный': 484, 'подержать': 478, 'приобретение': 459, 'профильный': 435})


Выберем уникальные цели получения кредита , которые встречаются чаще всего и не дублируют другие по смыслу.

#### Выбор уникальных целей получения кредита
Мной были выбраны следующие уникальные цели:
<ul>
    <li>недвижимость - все цели, относящиеся к недвижимости</li>
    <li>жильё - это также относится к недвижимости, поэтому жилью мы дадим тот же id, что и недвижимости</li>
    <li>автомобиль - все цели, связанные с авто</li>
    <li>образование - все цели, связанные с образованием</li>
    <li>свадьба - все цели, связанные со свадьбой</li>
</ul>
Создадим словарь типа 'ключ: значение', для удобства дальнейшей категоризации данных

In [31]:
purposes_dict = {'недвижимость': 1, 'жилье': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}
print(purposes_dict.keys())

dict_keys(['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'])


**Вывод**

На данном этапе была проведена лемматизация списка причин получения кредита и выделены уникальные цели.

### Категоризация данных

Сгруппируем данные по категориям. Категоризируем по следующим столбцам, что бы в дальнейшем ответить на поставленные вопросы:
<ul>
    <li>количество детей</li>
    <li>семейное положение</li>
    <li>уровень дохода</li>
    <li>цель кредита</li>
</ul>

#### Количество детей
Категоризируем данные по количеству детей:
<ul>
    <li>0 детей - бездетные</li>
    <li>1 - 2 детей - малодетные</li>
    <li>3 и более детей - многодетные</li>
</ul>
Напишем функцию, которая в зависимости от количества детей возвращает категорию.

In [32]:
def get_children_category(value):
    if value == 0:
        return 'бездетные'
    elif value >= 1 and value <= 2:
        return 'малодетные'
    else:
        return 'многодетные'

Применим данную функцию к новому столбцу, с категорией по количеству детей.

In [33]:
clients_data['children_category'] = clients_data['children'].apply(get_children_category)
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose,children_category
0,0,359219,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость,бездетные
1,0,369708,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием,бездетные
2,1,3642,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля,малодетные
3,0,359726,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,бездетные
4,0,346602,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,бездетные
5,0,347356,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью,бездетные
6,0,370478,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844,покупка жилья для семьи,бездетные
7,0,340952,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью,бездетные
8,0,4959,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование,бездетные
9,1,354563,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование,малодетные


Столбец с категорией по количеству детей выглядит верным, посмотрим на количество значений в столбце и убедимся, что все строки верны.

In [34]:
print(clients_data['children_category'].value_counts())

бездетные      14021
малодетные      6877
многодетные      453
Name: children_category, dtype: int64


Аномальных значений нет и все они верны.

#### Семейное положение
Посмотрим количество людей с разным семейным положением.

In [35]:
print(clients_data['family_status'].value_counts())

женат / замужем          12289
гражданский брак          4129
не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64


В этих данных также всё хорошо, был только исправлен формат написание и приведён к нижнему регистру.

#### Уровень дохода

##### Опредление диапазонов зарплат
Для начала, надо разобраться в том, какие вообще у клиентов бывают зарплаты за месяц, что бы понять, на какие диапазоны разбить ежемесячных доход.
Отсортируем данные по ежемесячной зарплате:

In [36]:
sorted_data = clients_data.sort_values(by='monthly_income')['monthly_income']
min_income = sorted_data.min()
max_income = sorted_data.max()
median_income = sorted_data.median()
print('Минимальная зарплата:', min_income)
print('Максимальная зарплата:', max_income)
print('Медиана зарплаты:', median_income)

Минимальная зарплата: 20667
Максимальная зарплата: 2265604
Медиана зарплаты: 143707.0


Возьмём 75% - 200% медианного дохода по всем клиентам (***по аналогии с национальным доходом по версии "Организации экономического сотрудничества и развития"***) - это будем считать за "средний" доход. Всё, что ниже - это будут клиенты с "низким" доходом, всё, что выше - "высокий" доход.<br>
Добавим также категорию "сверхвысокий доход", в котором будут клиенты, с доходои более миллиона, что бы выделить их в отдельную категорию.

In [37]:
print('Низкий доход: до', int(median_income * 0.75))
print('Средний доход: от', int(median_income * 0.75), 'до', int(median_income * 2))
print('Высокий доход: от', int(median_income * 2), 'до 1000000')
print('Сверхвысокий доход: от 1000000')

Низкий доход: до 107780
Средний доход: от 107780 до 287414
Высокий доход: от 287414 до 1000000
Сверхвысокий доход: от 1000000


##### Применение выделенных категорий
Напишем функцию, которая принимает в качестве параметра доход клиента и возвращает название категории, к которой относится клиент по уровню ежемесячной зарплаты.

In [38]:
def get_income_category(value):
    if value < 107780:
        return 'низкий доход'
    elif (value >= 107780) & (value < 287414):
        return 'средний доход'
    elif (value >= 287414) & (value < 1000000):
        return 'высокий доход'
    else:
        return 'сверхвысокий доход'

Применим функцию к столбцу, который будет содержать категорию клиента по его доходу.

In [39]:
clients_data['monthly_income_category'] = clients_data['monthly_income'].apply(get_income_category)
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose,children_category,monthly_income_category
0,0,359219,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость,бездетные,низкий доход
1,0,369708,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием,бездетные,низкий доход
2,1,3642,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля,малодетные,низкий доход
3,0,359726,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,бездетные,низкий доход
4,0,346602,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,бездетные,низкий доход
5,0,347356,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью,бездетные,низкий доход
6,0,370478,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844,покупка жилья для семьи,бездетные,низкий доход
7,0,340952,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью,бездетные,низкий доход
8,0,4959,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование,бездетные,низкий доход
9,1,354563,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование,малодетные,низкий доход


Посмотрим на количество новых данных.

In [40]:
print(clients_data['monthly_income_category'].value_counts())

средний доход         14294
низкий доход           5355
высокий доход          1677
сверхвысокий доход       25
Name: monthly_income_category, dtype: int64


Теперь все клиенты разделены на категории по уровню дохода.

#### Цель кредита
В пункте 2.5.1 нами были выделены леммы целей, а в пункте 2.5.2 выбраны уникальные цели, посмотрим на них ещё раз.

In [41]:
print(purposes_dict.keys())

dict_keys(['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'])


Напишем функцию, которая принимает в качестве параметра очередную строку, выделяет леммы цели кредита и проверяет, содержит ли список лемм какое либо слово из словаря с уникальными целями получения кредита, возращая её.

In [42]:
def get_purpose_category(purpose):
    row_lemmas = m.lemmatize(purpose)
    for lemma in row_lemmas:
        for category in purposes_dict.keys():
            if category in lemma:
                return purposes_dict.get(category)

Применим функцию к новому столбцу, который будет содержать id категории.

In [43]:
clients_data['purpose_category'] = clients_data['purpose'].apply(get_purpose_category)
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose,children_category,monthly_income_category,purpose_category
0,0,359219,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость,бездетные,низкий доход,1
1,0,369708,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием,бездетные,низкий доход,3
2,1,3642,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля,малодетные,низкий доход,2
3,0,359726,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,бездетные,низкий доход,4
4,0,346602,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,бездетные,низкий доход,1
5,0,347356,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью,бездетные,низкий доход,1
6,0,370478,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844,покупка жилья для семьи,бездетные,низкий доход,1
7,0,340952,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью,бездетные,низкий доход,1
8,0,4959,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование,бездетные,низкий доход,3
9,1,354563,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование,малодетные,низкий доход,3


Новый столбец заполнен. Только при анализе такого столбца, циферное обозначение неудобно. Заменим числа на понятный словестный формат:
<ul>
    <li>1 - недвижимость и жильё</li>
    <li>2 - авто</li>
    <li>3 - образование</li>
    <li>4 - свадьба</li>
</ul>

In [44]:
clients_data['purpose_category'] = clients_data['purpose_category'].replace(1, 'недвижимость и жильё')
clients_data['purpose_category'] = clients_data['purpose_category'].replace(2, 'авто')
clients_data['purpose_category'] = clients_data['purpose_category'].replace(3, 'образование')
clients_data['purpose_category'] = clients_data['purpose_category'].replace(4, 'свадьба')
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,work_type,credit_debt,monthly_income,purpose,children_category,monthly_income_category,purpose_category
0,0,359219,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость,бездетные,низкий доход,недвижимость и жильё
1,0,369708,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием,бездетные,низкий доход,образование
2,1,3642,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля,малодетные,низкий доход,авто
3,0,359726,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,бездетные,низкий доход,свадьба
4,0,346602,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,бездетные,низкий доход,недвижимость и жильё
5,0,347356,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью,бездетные,низкий доход,недвижимость и жильё
6,0,370478,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844,покупка жилья для семьи,бездетные,низкий доход,недвижимость и жильё
7,0,340952,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью,бездетные,низкий доход,недвижимость и жильё
8,0,4959,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование,бездетные,низкий доход,образование
9,1,354563,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование,малодетные,низкий доход,образование


Теперь категории о целе получения кредита понятны. Посмотрим на количество данных:

In [45]:
print(clients_data['purpose_category'].value_counts())

недвижимость и жильё    10762
авто                     4284
образование              3995
свадьба                  2310
Name: purpose_category, dtype: int64


Некорректных данных нет.

#### Проверка таблицы после добавления категорий
Проверим информацию о таблицы, чтобы убедиться, что после всех действий в таблице не появилось NaN значений.

In [46]:
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21351 entries, 0 to 21350
Data columns (total 15 columns):
children                   21351 non-null int64
days_employed              21351 non-null int64
dob_years                  21351 non-null int64
education                  21351 non-null object
education_id               21351 non-null int64
family_status              21351 non-null object
family_status_id           21351 non-null int64
gender                     21351 non-null object
work_type                  21351 non-null object
credit_debt                21351 non-null int64
monthly_income             21351 non-null int64
purpose                    21351 non-null object
children_category          21351 non-null object
monthly_income_category    21351 non-null object
purpose_category           21351 non-null object
dtypes: int64(7), object(8)
memory usage: 2.4+ MB


**Вывод**

На данном этапе мы добавли в таблицу следующие столбцы для дальнейшего анализа:
<ul>
    <li>По количеству детей</li>
    <li>По доходу</li>
    <li>По цели получения кредита</li>
</ul>

Также проверили стобец с семейным положением клиента.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
children_final = clients_data.groupby('children_category')['credit_debt'].agg(['count', 'sum'])
children_final['percent'] = (children_final['sum'] / children_final['count']) * 100
print(children_final)

                   count   sum   percent
children_category                       
бездетные          14021  1058  7.545824
малодетные          6877   635  9.233677
многодетные          453    39  8.609272


**Вывод**

В данном случае, можно сделать вывод, что чаще всех, кредит невозвращают семьи с 1-2 детьми, а реже всех - бездетные семьи.<br>
Такие результаты можно объяснить тем, что семьи, в которых 1-2 ребёнка:
<ul>
    <li>имеют повышенные траты</li>
    <li>доход делится на всех членов семьи</li>
</ul>
В то же время, процент клиентов, имевших задолженности по возврату кредита у многодетных семей меньше может быть по причине того, что такие семьи обладают льготами.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
family_final = clients_data.groupby('family_status')['credit_debt'].agg(['count', 'sum'])
family_final['percent'] = (family_final['sum'] / family_final['count']) * 100
print(family_final)

                       count  sum   percent
family_status                              
в разводе               1185   85  7.172996
вдовец / вдова           954   62  6.498952
гражданский брак        4129  386  9.348511
женат / замужем        12289  926  7.535194
не женат / не замужем   2794  273  9.770938


**Вывод**

В случае с семейным положение, задолженности по кредиту чаще остальных у людей, несостоящих в браке, так как данные люди не имеют финансовой поддержки в браке. А также, возможно, они менее ответственные.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
income_final = clients_data.groupby('monthly_income_category')['credit_debt'].agg(['count', 'sum'])
income_final['percent'] = (income_final['sum'] / income_final['count']) * 100
print(income_final)

                         count   sum   percent
monthly_income_category                       
высокий доход             1677   118  7.036374
низкий доход              5355   424  7.917834
сверхвысокий доход          25     2  8.000000
средний доход            14294  1188  8.311180


**Вывод**

В случае с уровнем дохода, задолженности по кредиту чаще у людей со средним доходом. Это можно объяснить неправильным планированием ежемесячных расходов и необдуманным сценарием выплаты кредита.

### Как разные цели кредита влияют на его возврат в срок?

In [50]:
purpose_final = clients_data.groupby('purpose_category')['credit_debt'].agg(['count', 'sum'])
purpose_final['percent'] = (purpose_final['sum'] / purpose_final['count']) * 100
print(purpose_final)

                      count  sum   percent
purpose_category                          
авто                   4284  400  9.337068
недвижимость и жильё  10762  778  7.229140
образование            3995  370  9.261577
свадьба                2310  184  7.965368


**Вывод**

В случае с целью кредита, задолженности по кредиту чаще в случае с автомобилями. Это можно объяснить следующими факторами:
- ДТП. В таком случае дополнительные расходы уходят на ремонт автомобиля
- при покупке поддержанного автомобиля могут потребоваться дополнительные расходы на его ремонт

## Шаг 4. Общий вывод

В результате проведённой работы, было сделано следующее:
- получены данные о клиентах кредитного отдела банка
- проведён первичный анализ данных и выявлены проблемы
- обработаны пропуски в данных
- обработаны ошибочные значения
- заменены типы данных
- обработаны дубликаты
- проведена лемматизация и выделение уникальных целей получения кредита
- проведена категоризация данных по количеству детей, семейному положению, уровню дохода и целью кредита
- проведено исследование на зависимость возврата кредита в срок от разных условий